In [8]:
import numpy as np
import csv

def clean_and_center_signal(signal, target_length):
    """
    signal: список значений (например, одна строка из CSV), представленных числами
    target_length: целевая длина сигнала (например, 65537)
    
    Возвращает:
        numpy array длины target_length или None (если невозможно обработать)
    """
    # 1. Удаляем пустые и пробельные элементы
    cleaned = [x for x in signal if x != '']
    
    if len(cleaned) == 0:
        return None
    
    # 2. Преобразуем в числа
    try:
        signal_array = np.array(cleaned, dtype=np.float32)
    except ValueError:
        return None  # битые данные
    
    L = len(signal_array)
    
    # 3. Обработка длины
    if L == target_length:
        return signal_array
    
    elif L > target_length:
        # Сигнал длиннее → обрезаем симметрично (с краёв)
        excess = L - target_length
        start = excess // 2
        end = start + target_length
        return signal_array[start:end]
    
    else:
        # Сигнал короче → дополняем (редкий случай)
        # Дополним средним значением (можно заменить на 0)
        pad = target_length - L
        pad_left = pad // 2
        pad_right = pad - pad_left
        mean_val = signal_array.mean() if L > 0 else 0.0
        return np.pad(signal_array, (pad_left, pad_right), mode='constant', constant_values=mean_val)

# Указываем тот же файл и строку
filename = "test.csv"
row_index = 41  # та же строка, которую ты аугментировал

# Читаем нужную строку
with open(filename, 'r', encoding='cp1251') as f:
    reader = csv.reader(f, delimiter=';')
    for i, row in enumerate(reader):
        if i == row_index:
            # Преобразуем строку в список чисел, исключая пустые элементы
            signal_from_file = [x for x in row if x.strip() != '']
            break

# Прогоняем через функцию
center_signal = clean_and_center_signal(signal_from_file, 65536)


In [14]:
from scipy.signal import savgol_filter

def smooth_signal(signal, window=101, polyorder=3):
    """
    Применяет фильтр Савицкого-Голея к одному сигналу.
    
    Параметры:
        signal: numpy array (T,)
        window: размер окна (нечётное число, чем больше — сильнее сглаживание)
        polyorder: степень полинома (обычно 2-3)
    """
    if window % 2 == 0:
        window += 1  # должно быть нечётным
    
    # Если сигнал короче окна — уменьшаем окно
    if len(signal) < window:
        window = len(signal) - 1 if len(signal) % 2 == 0 else len(signal)
        if window <= polyorder:
            return signal  # не фильтруем, если слишком мало точек
    
    return savgol_filter(signal, window_length=window, polyorder=polyorder)

In [27]:
import torch
import torch.nn as nn
import numpy as np

# Архитектура модели
class SignalCNN(nn.Module):
    def __init__(self):
        super().__init__()
        self.conv1 = nn.Conv1d(1, 32, kernel_size=7, padding=3)
        self.pool1 = nn.MaxPool1d(2)
        self.conv2 = nn.Conv1d(32, 64, kernel_size=5, padding=2)
        self.pool2 = nn.MaxPool1d(2)
        self.conv3 = nn.Conv1d(64, 128, kernel_size=3, padding=1)
        self.pool3 = nn.MaxPool1d(2)
        self.global_pool = nn.AdaptiveAvgPool1d(1)
        self.flatten = nn.Flatten()
        self.dropout = nn.Dropout(0.5)
        self.fc1 = nn.Linear(128, 64)
        self.fc2 = nn.Linear(64, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = torch.relu(self.conv1(x))
        x = self.pool1(x)
        x = torch.relu(self.conv2(x))
        x = self.pool2(x)
        x = torch.relu(self.conv3(x))
        x = self.pool3(x)
        x = self.global_pool(x)
        x = self.flatten(x)
        x = self.dropout(x)
        x = torch.relu(self.fc1(x))
        x = self.sigmoid(self.fc2(x))
        return x

# Функция для предсказания сигнала
def predict_signal_cnn(signal, model_weights_path="signal_cnn.pth"):
    # Строим модель
    model = SignalCNN()
    
    # Загружаем веса модели
    model.load_state_dict(torch.load(model_weights_path, map_location=torch.device('cuda')))
    model.eval()  # Важно! Устанавливаем модель в режим оценки
    
    # Применяем даунсэмплинг, что и при обучении (например, ::64)
    signal_down = signal[::64]  # → (1024,)
    
    # Преобразуем сигнал в тензор (1, 1, 1024)
    input_tensor = torch.tensor(signal_down, dtype=torch.float32).unsqueeze(0).unsqueeze(0)
    
    # Предсказание без вычисления градиентов
    with torch.no_grad():
        output = model(input_tensor)
        prob = output.item()  # вероятность класса "1"
        pred = 1 if prob > 0.5 else 0
    
    # Выводим результаты
    print("✅ Модель CNN")
    print(f"Вероятность сигнала: {prob:.4f}")
    print(f"Класс: {'сигнал есть' if pred == 1 else 'фон'}")

# Пример использования
# signal = np.array([...])  # Ваш сигнал длиной 65536
# predict_signal(signal)


In [29]:
import csv
import numpy as np
from scipy import stats
from scipy.fft import fft
from xgboost import XGBClassifier

# Функции извлечения признаков из сигнала
def extract_features(x):
    """
    Извлекает признаки из одного сигнала x (длина ~65000)
    Возвращает: numpy array формы (N_features,)
    """
    L = len(x)

    # Центральный участок (10% от длины)
    center_len = L // 10
    center_start = L // 2 - center_len // 2
    center_end = center_start + center_len
    center = x[center_start:center_end]

    # Крайние участки (по 5% с каждого края)
    edge_len = L // 20
    left_edge = x[:edge_len]
    right_edge = x[-edge_len:]
    edges = np.concatenate([left_edge, right_edge])

    features = []

    # --- Основные статистики ---
    features.append(np.mean(x))           # среднее
    features.append(np.std(x))            # стандартное отклонение
    features.append(np.max(x))            # максимум
    features.append(np.min(x))            # минимум
    features.append(np.ptp(x))            # размах
    features.append(np.mean(x**2))        # энергия
    features.append(np.mean(np.abs(x)))   # средняя абсолютная амплитуда

    # --- Центральные признаки ---
    features.append(np.mean(center))      # среднее в центре
    features.append(np.std(center))       # дисперсия в центре
    features.append(np.max(center))       # пик в центре
    features.append(np.mean(center**2))   # энергия в центре
    features.append(np.mean(center**2) / (np.mean(edges**2) + 1e-8))  # отношение центр/края

    # --- Дополнительные признаки ---
    features.append(stats.skew(x))        # асимметрия
    features.append(stats.kurtosis(x))    # эксцесс (острота)
    features.append(np.sum(np.abs(np.diff(x)) > 0.1))  # число "скачков" (приблизительно)

    # --- FFT-признаки (для гармоники) ---
    # Берём только первые 100 частот (все слишком много)
    fft_vals = np.abs(fft(x))[:100]
    features.append(np.max(fft_vals))     # доминирующая частота
    features.append(np.mean(fft_vals[1:10]))  # энергия в низких частотах
    features.append(np.mean(fft_vals[10:50])) # средние частоты
    features.append(np.mean(fft_vals[50:100])) # высокие частоты

    return np.array(features)

# Функция для предсказания с использованием XGBoost
def predict_signal_xgboost(X_new_feat, model_weights_path="xgboost_signal_classifier.json"):
    # Создаем модель XGBoost
    model = XGBClassifier()
    
    # Загружаем модель
    model.load_model(model_weights_path)
    print("✅ Модель XGBoost")
    
    # Предсказание классов
    y_pred = model.predict(X_new_feat)
    
    # Предсказание вероятности для класса 1
    y_proba = model.predict_proba(X_new_feat)[:, 1]  # вероятности класса 1
    
    # Выводим результаты
    for i in range(len(X_new_feat)):
        print(f"Прогноз - {'сигнал есть' if y_pred[i] == 1 else 'фон'}, Вероятность сигнала: {y_proba[i]:.4f}")
    


In [2]:
def count_lines(filename, encoding='cp1251'):
    with open(filename, encoding=encoding) as f:
        return sum(1 for _ in f)

total = count_lines('test.csv')
print(f"Всего строк в файле: {total}")

Всего строк в файле: 68


In [ ]:
def load_and_align_file(filename, target_length, delimiter=';', encoding='cp1251'):
    """
    Загружает файл, пропускает заголовок, обрабатывает каждую строку.
    Возвращает: np.array формы (N_valid, target_length)
    """
    signals = []
    
    with open(filename, 'r', encoding=encoding) as f:
        lines = f.readlines()
    
    for line_idx, line in enumerate(lines[1:], start=2):  # пропускаем заголовок
        line = line.strip()
        if not line:
            continue
        
        parts = line.split(delimiter)
        sig = clean_and_center_signal(parts, target_length)
        
        if sig is not None:
            signals.append(sig)
        else:
            print(f"⚠️ Пропущена строка {line_idx} в {filename}")
    
    return np.array(signals, dtype=np.float32) if signals else np.empty((0, target_length))

In [30]:
import csv
import numpy as np

# Указываем тот же файл и строку
filename = "test.csv"
row_index = 41  # та же строка, которую ты аугментировал

# Читаем нужную строку
with open(filename, 'r', encoding='cp1251') as f:
    reader = csv.reader(f, delimiter=';')
    for i, row in enumerate(reader):
        if i == row_index:
            signal_from_file = np.array([float(x) for x in row if x.strip() != ''])
            break
center_signal = clean_and_center_signal(signal_from_file, 65536)
smth_signal = smooth_signal(center_signal)
print 
predict_signal_cnn(smth_signal)
features = extract_features(smth_signal)

# Прогоняем через модель XGBoost
predict_signal_xgboost(features.reshape(1, -1))

✅ Модель CNN
Вероятность сигнала: 1.0000
Класс: сигнал есть
✅ Модель XGBoost
Прогноз - сигнал есть, Вероятность сигнала: 0.9901
